<a href="https://colab.research.google.com/github/brunomsm0/Data-Sciense/blob/main/Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [4]:
df = pd.read_excel('churn_data.xlsx')
df.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer,42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

In [6]:
df = df.dropna()

In [7]:
cat_vars = ["PhoneService", "Contract", "PaperlessBilling", "PaymentMethod"]
num_vars = ["tenure", "MonthlyCharges", "TotalCharges"]
modeling_vars = cat_vars + num_vars

In [8]:
X = df.filter(modeling_vars).copy()
y = df["Churn"].copy()
y = pd.DataFrame(y)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(y["Churn"])
y.Churn = le.transform(y.Churn)

In [10]:
X = pd.get_dummies(X)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify= y)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier()

parameters = {
    "max_depth": [1,2,3,4,5,6,7,8,9,10]
}

grid_search = GridSearchCV(arvore, parameters, scoring="roc_auc", cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='roc_auc')

In [14]:
results = pd.DataFrame(grid_search.cv_results_)


In [15]:
results.sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.022857,0.004329,0.016494,0.006177,4,{'max_depth': 4},0.821186,0.838586,0.817507,0.805943,0.824703,0.821585,0.010585,1
4,0.020715,0.000631,0.013266,0.000612,5,{'max_depth': 5},0.814895,0.816317,0.823903,0.813235,0.821184,0.817907,0.004003,2
5,0.022982,0.001381,0.012767,0.000553,6,{'max_depth': 6},0.809666,0.813215,0.817193,0.809219,0.817302,0.813319,0.003494,3
2,0.016270,0.001432,0.012946,0.000575,3,{'max_depth': 3},0.801384,0.833533,0.791055,0.798558,0.816400,0.808186,0.015115,4
6,0.025638,0.001167,0.012807,0.000977,7,{'max_depth': 7},0.794744,0.799390,0.806561,0.800647,0.796465,0.799561,0.004074,5
1,0.013007,0.000729,0.013361,0.000604,2,{'max_depth': 2},0.775102,0.803815,0.774088,0.769259,0.788809,0.782215,0.012606,6
7,0.027339,0.000470,0.013128,0.000236,8,{'max_depth': 8},0.785324,0.786803,0.792063,0.780357,0.765378,0.781985,0.009104,7
8,0.029725,0.000872,0.013704,0.000609,9,{'max_depth': 9},0.763787,0.771421,0.775535,0.772387,0.754590,0.767544,0.007540,8
9,0.029908,0.003850,0.012408,0.002448,10,{'max_depth': 10},0.734541,0.733770,0.762293,0.754195,0.731369,0.743233,0.012564,9
0,0.011606,0.001470,0.015859,0.004277,1,{'max_depth': 1},0.717501,0.748751,0.737476,0.723554,0.734951,0.732447,0.010953,10


In [16]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

parameters = {
    "max_depth": [1,2,3,4,5,6,7,8,9,10],
    "n_estimators": [100,300,500]
}

grid_search = GridSearchCV(rf, parameters, scoring="roc_auc", cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100, 300, 500]},
             scoring='roc_auc')

In [17]:
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
19,1.938182,0.036824,0.095609,0.002802,7,300,"{'max_depth': 7, 'n_estimators': 300}",0.834041,0.858752,0.840544,0.833308,0.837397,0.840809,0.009335,1
17,3.431828,0.607965,0.198315,0.057483,6,500,"{'max_depth': 6, 'n_estimators': 500}",0.833774,0.860406,0.838287,0.834357,0.835239,0.840413,0.010117,2
20,4.281063,0.912714,0.208104,0.064927,7,500,"{'max_depth': 7, 'n_estimators': 500}",0.833446,0.857898,0.840054,0.833013,0.837381,0.840358,0.009147,3
15,1.481129,0.446620,0.131925,0.058248,6,100,"{'max_depth': 6, 'n_estimators': 100}",0.833659,0.859953,0.836898,0.834839,0.835210,0.840112,0.009975,4
16,2.523211,0.922775,0.096417,0.005882,6,300,"{'max_depth': 6, 'n_estimators': 300}",0.832766,0.859070,0.838212,0.833938,0.834929,0.839783,0.009813,5
22,2.064080,0.016674,0.107910,0.011940,8,300,"{'max_depth': 8, 'n_estimators': 300}",0.832582,0.856349,0.840931,0.831013,0.837620,0.839699,0.009046,6
23,4.447377,0.900585,0.226601,0.064427,8,500,"{'max_depth': 8, 'n_estimators': 500}",0.833132,0.855056,0.841495,0.830853,0.837495,0.839606,0.008547,7
18,1.092851,0.189588,0.054444,0.011265,7,100,"{'max_depth': 7, 'n_estimators': 100}",0.833596,0.856934,0.837301,0.832889,0.835478,0.839240,0.008980,8
21,0.705084,0.014237,0.042445,0.001322,8,100,"{'max_depth': 8, 'n_estimators': 100}",0.832733,0.855631,0.840785,0.828614,0.837079,0.838968,0.009280,9
14,5.095901,1.077406,0.201043,0.052821,5,500,"{'max_depth': 5, 'n_estimators': 500}",0.831972,0.859896,0.833683,0.832337,0.834527,0.838483,0.010746,10


In [19]:
grid_search.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [20]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_jobs=-1)

In [21]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    "learning_rate": [0.01, 0.0],
    "num_leaves": [2,128],
    "min_child_samples": [1,100],
    "subsample": [0.05,1],
    "colsample_bytree": [0.1, 1.0]}


random_search = RandomizedSearchCV(lgbm, parameters, scoring="roc_auc", cv=5, n_jobs=-1, n_iter=10)

random_search.fit(X_train, y_train)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1495, number of negative: 4129
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000296 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 605
[LightGBM] [Info] Number of data points in the train set: 5624, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265825 -> initscore=-1.015909
[LightGBM] [Info] Start training from score -1.015909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py", line 1268, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py", line 954, in fit
    self._Booster = train(
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py", line 282, in train
    boost

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(n_jobs=-1), n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 1.0],
                                        'learning_rate': [0.01, 0.0],
                                        'min_child_samples': [1, 100],
                                        'num_leaves': [2, 128],
                                        'subsample': [0.05, 1]},
                   scoring='roc_auc')

In [22]:
results = pd.DataFrame(random_search.cv_results_)
results.sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_num_leaves,param_min_child_samples,param_learning_rate,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,1.564454,1.296727,0.042392,0.016316,0.05,128,100,0.01,1.0,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.827391,0.852885,0.842449,0.828838,0.831866,0.836686,0.009663,1
5,0.325094,0.182386,0.035896,0.014580,0.05,128,100,0.01,0.1,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",0.821238,0.850405,0.813936,0.819959,0.821206,0.825349,0.012816,2
1,1.109914,0.664765,0.022712,0.007171,1,2,1,0.01,0.1,"{'subsample': 1, 'num_leaves': 2, 'min_child_s...",0.823145,0.844648,0.802283,0.817701,0.819376,0.821431,0.013614,3
7,0.274239,0.118434,0.060282,0.011307,1,2,100,0.01,0.1,"{'subsample': 1, 'num_leaves': 2, 'min_child_s...",0.823145,0.844648,0.802283,0.817701,0.819376,0.821431,0.013614,3
8,3.360138,1.906727,0.030075,0.007233,1,128,1,0.01,1.0,"{'subsample': 1, 'num_leaves': 128, 'min_child...",0.805425,0.813968,0.826439,0.814780,0.819064,0.815935,0.006867,5
4,0.167607,0.050077,0.036005,0.012436,1,2,100,0.01,1.0,"{'subsample': 1, 'num_leaves': 2, 'min_child_s...",0.797580,0.832794,0.781953,0.804212,0.809628,0.805233,0.016616,6
0,0.022472,0.016199,0.000000,0.000000,1,128,1,0.0,0.1,"{'subsample': 1, 'num_leaves': 128, 'min_child...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,0.014130,0.003323,0.000000,0.000000,0.05,2,100,0.0,1.0,"{'subsample': 0.05, 'num_leaves': 2, 'min_chil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
6,0.037481,0.002030,0.000000,0.000000,0.05,128,1,0.0,0.1,"{'subsample': 0.05, 'num_leaves': 128, 'min_ch...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
9,0.013600,0.004505,0.000000,0.000000,0.05,2,1,0.0,0.1,"{'subsample': 0.05, 'num_leaves': 2, 'min_chil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
